In [ ]:
"""
To make sure the fusion decomposition works
up to the part where we would stitch the sublimbs together into one limb


"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:
import skeleton_utils as sk
import soma_extraction_utils as sm
import trimesh_utils as tu
import trimesh
import numpy_utils as nu
import numpy as np
from importlib import reload
import networkx as nx
import time
import compartment_utils as cu
import networkx_utils as xu
import matplotlib_utils as mu
import neuron_utils as nru

#importing at the bottom so don't get any conflicts
import itertools
from tqdm_utils import tqdm

#for meshparty preprocessing
import meshparty_skeletonize as m_sk
import general_utils as gu
import compartment_utils as cu
from meshparty import trimesh_io
from copy import deepcopy


from neuron_utils import *


# Loading the Neuron

In [3]:
import trimesh_utils as tu
curent_neuron = tu.load_mesh_no_processing("/notebooks/test_neurons/multi_soma_example.off")
segment_id = 12345
description = "double_soma"

In [4]:
# import neuron_visualizations as nviz
# nviz.plot_objects(main_mesh=curent_neuron)

# Setting the Arguments that would be present inside a preprocessing function call

In [5]:
#predefined arguments for the Neuron constructor

decomposition_type="meshafterparty"
mesh_correspondence="meshparty" #meshafterparty_adaptive
distance_by_mesh_center=True #how the distance is calculated for mesh correspondence
meshparty_segment_size = 100
meshparty_n_surface_downsampling = 2
meshparty_adaptive_correspondence_after_creation=False
suppress_preprocessing_print=True
computed_attribute_dict=None
somas = None
branch_skeleton_data=None
combine_close_skeleton_nodes = True
combine_close_skeleton_nodes_threshold=700
ignore_warnings=True
suppress_output=False
calculate_spines=True
widths_to_calculate=["no_spine_median_mesh_center",
                "no_spine_mean_mesh_center"]
fill_hole_size=2000

In [6]:
#arguments for the preprocess neuron
mesh = curent_neuron
segment_id=segment_id
description=description

sig_th_initial_split=15 #for significant splitting meshes in the intial mesh split
limb_threshold = 2000 #the mesh faces threshold for a mesh to be qualified as a limb (otherwise too small)
filter_end_node_length=4001 #used in cleaning the skeleton during skeletonizations
return_no_somas = False

decomposition_type=decomposition_type
mesh_correspondence=mesh_correspondence
distance_by_mesh_center=distance_by_mesh_center
meshparty_segment_size =meshparty_segment_size
meshparty_n_surface_downsampling = meshparty_n_surface_downsampling
somas=somas
branch_skeleton_data=branch_skeleton_data
combine_close_skeleton_nodes = combine_close_skeleton_nodes
combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold

# Setting up the metadata

In [7]:
whole_processing_tiempo = time.time()


"""
Purpose: To process the mesh into a format that can be loaded into the neuron class
and used for higher order processing (how to visualize is included)

"""
if description is None:
    description = "no_description"
if segment_id is None:
    #pick a random segment id
    segment_id = np.random.randint(100000000)
    print(f"picking a random 7 digit segment id: {segment_id}")
    description += "_random_id"


if mesh is None:
    if current_mesh_file is None:
        raise Exception("No mesh or mesh_file file were given")
    else:
        current_neuron = trimesh.load_mesh(current_mesh_file)
else:
    current_neuron = mesh

# ************************ Phase A: Soma and Limb Identification ********************************

In [8]:
# --- 1) Doing the soma detection
if somas is None:
    soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)
else:
    soma_mesh_list,run_time,total_soma_list_sdf = somas

# geting the soma centers
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
    if return_no_somas:
        return_value= soma_mesh_list_centers
    raise Exception("Processing of No Somas is not yet implemented yet")
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = sm.find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 525 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/12345/neuron_12345.off -o /notebooks/Platinum_Decomp_Fusion/12345/neuron_12345_decimated.off -s /notebooks/Platinum_Decomp_Fusion/12345/decimation_meshlab_2553772.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284336, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284336, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Decomp_Fusion/12345/neuron_12345_decimated_largest_piece.off
xvfb-run -n 2898 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/12345/neuron_12345_decimated_largest_piece.off -o /notebooks/Plat

# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


----Working on soma-containing mesh piece 0----
current_soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(3065, 3), faces.shape=(6001, 3))>]
current_mesh = <trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284334, 3))>

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(57549, 3), faces.shape=(117088, 3))>, <trimesh.Trimesh(vertices.shape=(37282, 3), faces.shape=(75647, 3))>, <trimesh.Trimesh(vertices.shape=(14433, 3), faces.shape=(29649, 3))>, <trimesh.Trimesh(vertices.shape=(13548, 3), faces.shape=(27337, 3))>, <trimesh.Trimesh(vertices.shape=(12269, 3), faces.shape=(24920, 3))>, <trimesh.Trimesh(vertices.shape=(274, 3), faces.shape=(524, 3))>]
There were 6 pieces found after size threshold
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(57549, 3), faces.shape=(117088, 3))>, <trimesh.

In [9]:
#--- 2) getting the soma submeshes that are connected to each soma and identifiying those that aren't (and eliminating any mesh pieces inside the soma)

main_mesh_total = current_neuron


#finding the mesh pieces that contain the soma
#splitting the current neuron into distinct pieces
split_meshes = tu.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=sig_th_initial_split,
                            print_flag=False)

print(f"# total split meshes = {len(split_meshes)}")


#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = sm.find_soma_centroid_containing_meshes(soma_mesh_list,
                                        split_meshes)

# filtering away any of the inside floating pieces: 
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]


#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    """
    *** want to save these pieces that are inside of the soma***
    """

    non_soma_touching_meshes,inside_pieces = sm.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes,
                                    significance_threshold=sig_th_initial_split,
                                    return_inside_pieces = True)                                                      


split_meshes # the meshes of the original mesh
containing_mesh_indices #the mapping of each soma centroid to the correct split mesh
soma_containing_meshes = sm.grouping_containing_mesh_indices(containing_mesh_indices)

soma_touching_meshes = [split_meshes[k] for k in soma_containing_meshes.keys()]


#     print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
#     print(f"# of inside pieces = {len(inside_pieces)}")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")






# total split meshes = 79
There were 78 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0, 1]}


In [10]:
tu = reload(tu)
#--- 3)  Soma Extraction was great (but it wasn't the original soma faces), so now need to get the original soma faces and the original non-soma faces of original pieces

#     sk.graph_skeleton_and_mesh(other_meshes=[soma_meshes])

"""
for each soma touching mesh get the following:
1) original soma meshes
2) significant mesh pieces touching these somas
3) The soma connectivity to each of the significant mesh pieces
-- later will just translate the 


Process: 

1) Final all soma faces (through soma extraction and then soma original faces function)
2) Subtact all soma faces from original mesh
3) Find all significant mesh pieces
4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all
   the available somas
Conclusion: Will have connectivity map


"""

soma_touching_mesh_data = dict()

for z,(mesh_idx, soma_idxes) in enumerate(soma_containing_meshes.items()):
    soma_touching_mesh_data[z] = dict()
    print(f"\n\n----Working on soma-containing mesh piece {z}----")

    #1) Final all soma faces (through soma extraction and then soma original faces function)
    current_mesh = split_meshes[mesh_idx]

    current_soma_mesh_list = [soma_mesh_list[k] for k in soma_idxes]

    current_time = time.time()
    mesh_pieces_without_soma = sm.subtract_soma(current_soma_mesh_list,current_mesh,
                                                significance_threshold=250)
    print(f"Total time for Subtract Soam = {time.time() - current_time}")
    current_time = time.time()

    mesh_pieces_without_soma_stacked = tu.combine_meshes(mesh_pieces_without_soma)

    # find the original soma faces of mesh
    soma_faces = tu.original_mesh_faces_map(current_mesh,mesh_pieces_without_soma_stacked,matching=False)
    print(f"Total time for Original_mesh_faces_map for mesh_pieces without soma= {time.time() - current_time}")
    current_time = time.time()
    soma_meshes = current_mesh.submesh([soma_faces],append=True,repair=False)

    # finding the non-soma original faces
    non_soma_faces = tu.original_mesh_faces_map(current_mesh,soma_meshes,matching=False)
    non_soma_stacked_mesh = current_mesh.submesh([non_soma_faces],append=True,repair=False)

    print(f"Total time for Original_mesh_faces_map for somas= {time.time() - current_time}")
    current_time = time.time()

    # 3) Find all significant mesh pieces
    sig_non_soma_pieces,insignificant_limbs = tu.split_significant_pieces(non_soma_stacked_mesh,significance_threshold=limb_threshold,
                                                     return_insignificant_pieces=True)

    print(f"Total time for sig_non_soma_pieces= {time.time() - current_time}")
    current_time = time.time()

    soma_touching_mesh_data[z]["branch_meshes"] = sig_non_soma_pieces

    #4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all the available somas
    # get all the seperate mesh faces

    #How to seperate the mesh faces
    seperate_soma_meshes,soma_face_components = tu.split(soma_meshes,only_watertight=False)
    #take the top largest ones depending how many were originally in the soma list
    seperate_soma_meshes = seperate_soma_meshes[:len(soma_mesh_list)]
    soma_face_components = soma_face_components[:len(soma_mesh_list)]

    soma_touching_mesh_data[z]["soma_meshes"] = seperate_soma_meshes

    print(f"Total time for split= {time.time() - current_time}")
    current_time = time.time()



    soma_to_piece_connectivity = dict()
    soma_to_piece_touching_vertices = dict()
    soma_to_piece_touching_vertices_idx = dict()
    limb_root_nodes = dict()
    
    m_vert_graph = tu.mesh_vertex_graph(current_mesh)
    
    for i,curr_soma in enumerate(seperate_soma_meshes):
        (connected_mesh_pieces,
         connected_mesh_pieces_vertices,
         connected_mesh_pieces_vertices_idx) = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True,
                        return_vertices_idx=True)
        #print(f"soma {i}: connected_mesh_pieces = {connected_mesh_pieces}")
        soma_to_piece_connectivity[i] = connected_mesh_pieces

        soma_to_piece_touching_vertices[i] = dict()
        for piece_index,piece_idx in enumerate(connected_mesh_pieces):
            limb_root_nodes[piece_idx] = connected_mesh_pieces_vertices[piece_index][0]
            
            """ Old way of finding vertex connected components on a mesh without trimesh function
            #find the number of touching groups and save those 
            soma_touching_graph = m_vert_graph.subgraph(connected_mesh_pieces_vertices_idx[piece_index])
            soma_con_comp = [current_mesh.vertices[np.array(list(k)).astype("int")] for k in list(nx.connected_components(soma_touching_graph))]
            soma_to_piece_touching_vertices[i][piece_idx] = soma_con_comp
            """
            
            soma_to_piece_touching_vertices[i][piece_idx] = tu.split_vertex_list_into_connected_components(
                                                vertex_indices_list=connected_mesh_pieces_vertices_idx[piece_index],
                                                mesh=current_mesh, 
                                                vertex_graph=m_vert_graph, 
                                                return_coordinates=True
                                               )
            
            
            
            

#         border_debug = False
#         if border_debug:
#             print(f"soma_to_piece_connectivity = {soma_to_piece_connectivity}")
#             print(f"soma_to_piece_touching_vertices = {soma_to_piece_touching_vertices}")


    print(f"Total time for mesh_pieces_connectivity= {time.time() - current_time}")

    soma_touching_mesh_data[z]["soma_to_piece_connectivity"] = soma_to_piece_connectivity

print(f"# of insignificant_limbs = {len(insignificant_limbs)} with trimesh : {insignificant_limbs}")



# Lets have an alert if there was more than one soma disconnected meshes
if len(soma_touching_mesh_data.keys()) > 1:
    raise Exception("More than 1 disconnected meshes that contain somas")

current_mesh_data = soma_touching_mesh_data
soma_containing_idx = 0

#doing inversion of the connectivity and touching vertices
piece_to_soma_touching_vertices = gu.flip_key_orders_for_dict(soma_to_piece_touching_vertices)


# ****Soma Touching mesh Data has the branches and the connectivity (So this is where you end up skipping if you don't have somas)***



----Working on soma-containing mesh piece 0----

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(153780, 3), faces.shape=(308589, 3))>, <trimesh.Trimesh(vertices.shape=(87518, 3), faces.shape=(175900, 3))>, <trimesh.Trimesh(vertices.shape=(58482, 3), faces.shape=(117235, 3))>, <trimesh.Trimesh(vertices.shape=(56775, 3), faces.shape=(114294, 3))>, <trimesh.Trimesh(vertices.shape=(54856, 3), faces.shape=(109933, 3))>, <trimesh.Trimesh(vertices.shape=(51741, 3), faces.shape=(103848, 3))>, <trimesh.Trimesh(vertices.shape=(29266, 3), faces.shape=(58666, 3))>, <trimesh.Trimesh(vertices.shape=(29129, 3), faces.shape=(58497, 3))>, <trimesh.Trimesh(vertices.shape=(14705, 3), faces.shape=(29558, 3))>, <trimesh.Trimesh(vertices.shape=(1132, 3), faces.shape=(2218, 3))>]
There were 10 pieces found after size threshold
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(153780, 

/meshAfterParty/trimesh_utils.py:314: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


Total time for mesh_pieces_connectivity= 18.07354474067688
# of insignificant_limbs = 0 with trimesh : []


# Process that will start for each limb

In [29]:
sk = reload(sk)
tu = reload(tu)
cu = reload(cu)
m_sk = reload(m_sk)
pre = reload(pre)
xu = reload(xu)
nu = reload(nu)
gu = reload(gu)


proper_time = time.time()

#The containers that will hold the final data for the preprocessed neuron
limb_correspondence=dict()
limb_network_stating_info = dict()

# ---------- Part A: skeletonization and mesh decomposition --------- #
skeleton_time = time.time()

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):
    
    #Arguments to pass to the specific function (when working with a limb)
    soma_touching_vertices_dict = piece_to_soma_touching_vertices[curr_limb_idx]
    
#     if curr_limb_idx != 5:
#         continue
    
    curr_limb_time = time.time()
    print(f"\n\n----- Working on Proper Limb # {curr_limb_idx} ---------")
    

    limb_correspondence_individual,network_starting_info = pre.preprocess_limb(mesh=limb_mesh_mparty,
                   soma_touching_vertices_dict = soma_touching_vertices_dict,
                   return_concept_network = False, 
                   return_concept_network_starting_info=True,
                   width_threshold_MAP=500,
                   size_threshold_MAP=2000,
                   surface_reconstruction_size=1000,                                                            
                   )
    #Storing all of the data to be sent to 
    
    limb_correspondence[curr_limb_idx] = limb_correspondence_individual
    limb_network_stating_info[curr_limb_idx] = network_starting_info
    
#     raise Exception("Done with #5")
    
    
print(f"Total time for Skeletonization and Mesh Correspondence = {time.time() - skeleton_time}")

pre=reload(pre)
neuron=reload(neuron)
nru = reload(nru)
# ---------- Part B: Stitching on floating pieces --------- #
floating_stitching_time = time.time()

limb_correspondence_with_floating_pieces = pre.attach_floating_pieces_to_limb_correspondence(
        limb_correspondence,
        floating_meshes=non_soma_touching_meshes,
        floating_piece_face_threshold = 600,
        max_stitch_distance=8000,
        distance_to_move_point_threshold = 4000,
        verbose = False)



print(f"Total time for stitching floating pieces = {time.time() - floating_stitching_time}")





# ---------- Part C: Computing Concept Networks --------- #
concept_network_time = time.time()

limb_concept_networks=dict()
limb_labels=dict()

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):
    limb_to_soma_concept_networks = pre.calculate_limb_concept_networks(limb_correspondence_with_floating_pieces[curr_limb_idx],
                                                                        run_concept_network_checks=True,
                                                                       **limb_network_stating_info[curr_limb_idx])   



    limb_concept_networks[curr_limb_idx] = limb_to_soma_concept_networks
    limb_labels[curr_limb_idx]= "Unlabeled"
    
print(f"Total time for Concept Networks = {time.time() - concept_network_time}")





preprocessed_data= dict(
    soma_meshes = current_mesh_data[0]["soma_meshes"],
    soma_to_piece_connectivity = current_mesh_data[0]["soma_to_piece_connectivity"],
    soma_sdfs = total_soma_list_sdf,
    insignificant_limbs=insignificant_limbs,
    non_soma_touching_meshes=non_soma_touching_meshes,
    inside_pieces=inside_pieces,
    limb_correspondence=limb_correspondence_with_floating_pieces,
    limb_concept_networks=limb_concept_networks,
    limb_network_stating_info=limb_network_stating_info,
    limb_labels=limb_labels,
    limb_meshes=current_mesh_data[0]["branch_meshes"],
    )

    

print(f"Total time for all limb decomps = {time.time() - proper_time}")


neuron_obj = neuron.Neuron(
                mesh=curent_neuron,
                 segment_id=segment_id,
                 description=description,
                 preprocessed_data=preprocessed_data,
#                  decomposition_type="meshafterparty",
#                  mesh_correspondence="meshparty", #meshafterparty_adaptive
#                  distance_by_mesh_center=True, #how the distance is calculated for mesh correspondence
#                  meshparty_segment_size = 0,
#                  meshparty_n_surface_downsampling = 0,
#                  meshparty_adaptive_correspondence_after_creation=False,
#                 suppress_preprocessing_print=True,
#                  computed_attribute_dict=None,
#                  somas = None,
#                  branch_skeleton_data=None,
#                  combine_close_skeleton_nodes = True,
#                 combine_close_skeleton_nodes_threshold=700,
    
    
                ignore_warnings=True,
                suppress_output=False,
                calculate_spines=False,
                 #widths_to_calculate=["no_spine_median_mesh_center"]
                widths_to_calculate=[]
                )
#864049.29563888 1004924.982468    884750.28428994



----- Working on Proper Limb # 0 ---------
Time for preparing soma vertices and root: 1.1682510375976562e-05
cc_vertex_thresh = 10


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.35s/it]

Time for 1st pass MP skeletonization: 5.314494371414185
branches_touching_root = [60]
length of Graph = 15718
Working on path [7923. 7950. 7993. 8001.]
path_degrees = [3, 2, 2, 3]
kept_branches_idx = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60]
max(kept_branches_idx) = 60, len(kept_branches_idx) = 60
empty_indices % = 0.0
 conflict_indices % = 0.01305300270911377



AFTER face_lookup_resolved_test


...

all graph is one component!

Total time for skeleton stitching = 0.02085423469543457
The process was using a temp folder
Checking connected components after skeletonize_connected_branch
    Total time for skeletonizing branch: 5.839212417602539
Checking connected components after removing cycles
Total time for mesh KDTree = 0.025153160095214844
sbv[0].reshape(-1,3) = [[760911.2 964428.3 879199.1]]
closest_sk_pt_coord BEFORE = [761020. 964581. 878536.]
current_skeleton.shape = (24, 2, 3)
Current stitch point was not a branch or endpoint, shortest_path_length to one = 466.46225992596953
Changing the stitch point becasue the distance to end or branch node was 466.46225992596953
New stitch point has degree 1
change_status for create soma extending pieces = True
closest_sk_pt_coord AFTER = [761104. 964131. 878447.]
skipping soma 0 because closest skeleton node was already end node
endpoints_must_keep = {0: array([[761104., 964131., 878447.]])}
    Total time for Fixing Skeleton Soma Endpoin


Total time for decomposition = 0.18721652030944824
mesh_correspondence_first_pass: 0.18724679946899414
Limb decomposed into 1 branches
divided_skeleton_graph_recovered = (24, 2, 3) and 
current_mesh_data[0]['branch_skeletons_cleaned'].shape = (24, 2, 3)

Number of connected components in deocmposed recovered graph = 1
Number of connected components in cleaned skeleton graph= 1
The downsampled branches number of connected components = 1
Empty submeshes = []


--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (0, 1)
empty_indices % = 0.046438232642019836
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test
Took 0 iterations to expand the label back



Total time for MAP sublimb #0 mesh processing = 6.219240188598633
correspondence_1_to_1: 0.09488630294799805
Total time for MAP sublimb processing 6.219321966171265
There were not both MAP and MP pieces so skipping the stitch resolving phase
Time for decomp of Limb = 6.390238285064697
Total time for Skeletonization and Mesh Correspondence = 755.8467605113983
Status of Main limb stitch point moved = True
Total time for stitching floating pieces = 0.4561891555786133
Starting_edge inside branches_to_conept = [[755936.         946897.         874482.        ]
 [756963.73513514 950678.05765766 873676.06486486]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [37]
printing out current edge:
[[756963.73513514 950678.05765766 873676.06486486]
 [755936.         946897.         874482.        ]]
edge_endpoints_to_process was empty so exiting loop after 60 iterations
starting_node in concept map (that should match the starting edge) = 12
Total tim

--- 3) Finshed generating soma objects and adding them to concept graph: 19.20608425140381
--- 4a) Finshed generating curr_limb_meshes_face_idx: 3.597524881362915
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2bd30>]}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2bd30>]}
checking and resolving cycles
No cycles to fix
curr_limb_concept_networks= {1: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2b470>], 0: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2b208>]}
concept_network_dict = {1: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2b470>], 0: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2b208>]}
checking and resolving cycles
No cycles to fix
curr_limb_concept_networks= {1: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2b0b8>]}
concept_network_dict = {1: [<networkx_utils.GraphOrderedEdges object at 0x7fac42a2b0b8>]}
checking and resolving cycles
No cycles to fix
curr_limb

In [ ]:
# nviz.plot_objects(meshes=[k["branch_mesh"] for k in limb_correspondence_individual.values()],
#                   meshes_colors="random",
#                  skeletons=[k["branch_skeleton"] for k in limb_correspondence_individual.values()],
#                  skeletons_colors="random")

# Sending the data to the Neuron Object

In [ ]:
np.sum([len(k.faces) for k in divided_submeshes]),curr_limb.mesh

In [ ]:
nru = reload(nru)
save_time = time.time()
neuron_obj.save_compressed_neuron(output_folder="./",
                                 export_mesh=False,
                                 suppress_output=False)
print(f"Save time = {time.time() - save_time}")

In [ ]:
nru = reload(nru)
reload_time = time.time()
recovered_neuron = nru.decompress_neuron(filepath="./12345_double_soma",
                     original_mesh="./12345_double_soma")
print(f"Save time = {time.time() - reload_time}")

In [ ]:
import neuron_visualizations as nviz
returned_colors = nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict=dict(L6="all"),
                                       return_color_dict=True)

In [ ]:
curr_limb = neuron_obj[6]

In [ ]:
m_sk = reload(m_sk)
sk = reload(sk)

limb_mesh_mparty = curr_limb.mesh

sk_meshparty_obj = m_sk.skeletonize_mesh_largest_component(limb_mesh_mparty,
                                                            root=curr_limb.current_touching_soma_vertices[0],
                                                              filter_mesh=False)

(segment_branches, #skeleton branches
divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
segment_widths_median) = m_sk.skeleton_obj_to_branches(sk_meshparty_obj,
                                                      mesh = limb_mesh_mparty,
                                                      meshparty_segment_size=100)

In [ ]:
nviz.plot_objects(
                    #main_mesh = curr_limb.mesh,
                    meshes=divided_submeshes,
                  meshes_colors="random",
                 skeletons=segment_branches,
                 skeletons_colors="random"
)

In [ ]:
MAP_surround = [4,6]
small_connector = 5
t_cross = [7,12]

In [ ]:
sk.calculate_skeleton_distance(segment_branches[5])

In [ ]:
idx_to_plot = [5]
divided_submeshes = np.array(divided_submeshes)
segment_branches = np.array(segment_branches)
nviz.plot_objects(
                    main_mesh = curr_limb.mesh,
                    meshes=divided_submeshes[idx_to_plot],
                  meshes_colors="random",
                 skeletons=segment_branches[idx_to_plot],
                 skeletons_colors="random"
)

In [ ]:
skeleton_branches = sk.decompose_skeleton_to_branches(total_sk)
branch_idx_to_endpoints = np.array([sk.find_branch_endpoints(k) for k in skeleton_branches])
branch_idx_to_endpoints.shape

In [ ]:
sk = reload(sk)
total_sk = sk.stack_skeletons([MP_piece_skeleton,MAP_piece_skeleton])
total_sk

In [ ]:
nx.number_connected_components(sk.convert_skeleton_to_graph(total_sk))

In [ ]:
set(np.array([1,2,3,4,5]))

In [ ]:
sk = reload(sk)
xu = reload(xu)
total_sk_decomposed = sk.decompose_skeleton_to_branches(total_sk)
returned_branches,kept_branches = sk.combine_close_branch_points(skeleton_branches=total_sk_decomposed,
                                                                 combine_threshold = 700,
                                               print_flag=True)

In [ ]:
kept_branches

In [ ]:
total_sk_revised = sk.stack_skeletons(returned_branches)
nx.number_connected_components(sk.convert_skeleton_to_graph(total_sk_revised))

In [ ]:
nviz.plot_objects(meshes=[MP_piece,MAP_piece],
                 skeletons=[total_sk_revised])

In [ ]:
ex_coord = np.array([746919.52489596, 963059.04699111, 884002.79966543])

In [ ]:
orig_sk_gr = sk.convert_skeleton_to_graph(orig_sk)
xu.get_graph_node_by_coordinate(orig_sk_gr,ex_coord)

In [ ]:
idx_to_plot = [5,7,12]
MP_piece = tu.combine_meshes([divided_submeshes[k] for k in idx_to_plot])
MP_piece_skeleton = sk.stack_skeletons(segment_branches[idx_to_plot])
idx_to_plot = [4,6]
MAP_piece = tu.combine_meshes([divided_submeshes[k] for k in idx_to_plot])
MAP_piece_skeleton = sk.stack_skeletons(segment_branches[idx_to_plot])
nviz.plot_objects(meshes=[MP_piece,MAP_piece],
                 skeletons=[MP_piece_skeleton,MAP_piece_skeleton])

In [ ]:


nviz.plot_objects(meshes=[neuron_obj[6][k].mesh for k in idx_to_plot],
                 meshes_colors="random",
                 skeletons=[neuron_obj[6][k].skeleton for k in idx_to_plot],
                 skeletons_colors="random")

In [ ]:
neuron_obj[0][31].width,neuron_obj[0][33].width

In [ ]:
neuron = reload(neuron)
neuron_obj = neuron.Neuron(neuron_obj)
nviz.plot_limb_concept_network_2D(neuron_obj,
                                 node_colors=returned_colors,
                                  node_size=1000,
                                  font_size=15,
                                 pos_width=1.5)

In [ ]:
nviz.plot()

In [ ]:
neuron_obj[6][19].mesh,neuron_obj[6][20].mesh,neuron_obj[6][17].mesh,neuron_obj[6][18].mesh

In [ ]:
limb_correspondence[6][0]

In [ ]:
idx_to_plot = [17]
nviz.plot_objects(meshes=[limb_correspondence[6][k]["branch_mesh"] for k in idx_to_plot],
                  meshes_colors="random",
                  skeletons=[limb_correspondence[6][k]["branch_skeleton"]  for k in idx_to_plot],
                  skeletons_colors="random")

In [ ]:
idx_to_plot = [16]
nviz.plot_objects(meshes=[neuron_obj[6][k].mesh for k in idx_to_plot],
                  meshes_colors="random",
                  skeletons=[neuron_obj[6][k].skeleton for k in idx_to_plot],
                  skeletons_colors="random")

In [ ]:
import spine_utils as spu
tu = reload(tu)
spu = reload(spu)
neuron_obj.calculate_spines()

In [ ]:
nviz.plot_spines(neuron_obj)

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)

In [ ]:
neuron_obj.save_compressed_neuron("/notebooks/test_neurons/Fusion_decomp/",
                                 export_mesh=True)